In [ ]:
!pip install promptflow-evals

In [14]:
import os

import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

azure_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))

# Initialize Azure OpenAI client
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")

AZURE_SEARCH_FULL_INDEX = "dbpedia-1m-all"
search_client = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)

In [15]:
user_question = "What does a product manager do?"
user_question_vector = get_embedding(user_question)

# Perform the search query
r = search_client.search(
    user_question,
    top=5, 
    vector_queries=[
        VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="embedding")
    ],
    query_type="semantic",
    semantic_configuration_name="default"
)

# Format the sources to include id, title, and text
sources = "\n\n".join([f"[{doc['id']}]: {doc['title']} - {doc['text']}\n" for doc in r])

# Define the system message
SYSTEM_MESSAGE = """
Assistant helps users find relevant information from the DBpedia dataset. DBpedia is a structured version of Wikipedia, containing factual information.
Be concise in your answers, and answer ONLY with the facts listed in the sources below.
If there isn't enough information from the sources, respond by stating you don't know. Avoid generating answers not based on the sources below.
Each source has a name followed by the actual information. Always include the source name for each fact you use.
Use square brackets to reference the source, for example [doc1].
"""

# Define the user message
USER_MESSAGE = user_question + "\nSources: " + sources

# Generate the response using OpenAI
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME"),
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

# Print the answer
answer = response.choices[0].message.content
print(answer)

A product manager investigates, selects, and drives the development of products for an organization, performing the activities of product management. They consider factors such as the intended demographic, the products offered by the competition, and how well the product fits with the company's business model. Generally, a product manager manages one or more tangible products [4a5576f9-1604-4ca2-b1d2-464c30f3498c].


In [17]:
import os

from promptflow.core import AzureOpenAIModelConfiguration
from promptflow.evals.evaluators import GroundednessEvaluator, RelevanceEvaluator

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.getenv("AZURE_OEPNAI_ENDPOINT"),
    azure_deployment=os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME"),
)

relevance_eval = RelevanceEvaluator(model_config)
groundedness_eval = GroundednessEvaluator(model_config)

relevance_score = relevance_eval(
    question=user_question,
    answer=answer,
    context=sources,
)
print(relevance_score)

groundedness_score = groundedness_eval(
    answer=answer,
    context=sources,
)
print(groundedness_score)

{'gpt_relevance': 5.0}
{'gpt_groundedness': 5.0}
